## Map Mutations to 3D Structures in the Protein Data Bank

In [1]:
from pyspark.sql import SparkSession
from mmtfPyspark.datasets import g2sDataset, pdbjMineDataset
from ipywidgets import interact, IntSlider
import py3Dmol
import pandas as pd

#### Input parameters

In [2]:
distance_cutoff = 8 # distance cutoff for visualizing interactions
input_file_name = 'mutations.csv' # mutations in standard format (e.g., chr5:g.149440497C>T)
output_file_name = 'mutations3d.csv' # mutations mapped to 3D protein structures

## Read 'mutations.csv' file created in the previous step

In [3]:
df = pd.read_csv(input_file_name)

## Create a list of the variants

In [4]:
var_ids = df['var_id'].tolist()
var_ids

['chr6:g.52619766C>T', 'chr9:g.133738358A>T', 'chr11:g.5246945G>T']

## Map the mutations from genome locations to 3D PDB positions¶
Here we use [g2sDataset](https://github.com/sbl-sdsc/mmtf-pyspark/blob/master/mmtfPyspark/datasets/g2sDataset.py) to retrieve genome to PDB mapping information using the [G2S](https://g2s.genomenexus.org/) (Genome to Structure) web services.

Note, this step may take several minutes to complete.

In [5]:
pdb_map = g2sDataset.get_position_dataset(var_ids, ref_genome='hgvs-grch37').toPandas()

### Reduce redundant data to speed up this demo.
For you own data you may want to comment out this step to make sure you are getting all PDB mappings

In [6]:
pdb_map = pdb_map.drop_duplicates(subset=['structureId','pdbPosition','pdbAminoAcid','variationId'])

In [7]:
pdb_map.head()

,structureId,chainId,pdbPosition,pdbAminoAcid,refGenome,variationId
0,1GUH,A,83,G,hgvs-grch37,chr6:g.52619766C>T
1,1XZ7,B,108,N,hgvs-grch37,chr11:g.5246945G>T
2,2VCT,D,83,G,hgvs-grch37,chr6:g.52619766C>T
3,3KMF,G,708,N,hgvs-grch37,chr11:g.5246945G>T
4,6HD6,A,272,Y,hgvs-grch37,chr9:g.133738358A>T


In [8]:
# You can also get the mapping for a specific PDB chain, e.g.,

#pdb_map = g2sDataset.get_position_dataset(var_ids, '4ACS', 'B', ref_genome='hgvs-grch37').toPandas()

## Filter PDB Chains

Here we use the SIFTS annotation provided by EBI to filter by taxonomy. To learn more about how to [retrieve SIFTS annotation](
https://github.com/sbl-sdsc/mmtf-pyspark/blob/master/demos/datasets/SiftsDataDemo.ipynb).

#### To filter by organism, we first retrieve the SIFTS taxonomy annotations on a PDB chain level
Here we are only interested in human proteins.

In [9]:
taxonomyQuery = "SELECT * FROM sifts.pdb_chain_taxonomy WHERE sifts.pdb_chain_taxonomy.scientific_name = 'Homo sapiens'"
taxonomy = pdbjMineDataset.get_dataset(taxonomyQuery).toPandas()
taxonomy.head()

,pdbid,chain,tax_id,scientific_name,structureChainId
0,10GS,A,9606,Homo sapiens,10GS.A
1,10GS,B,9606,Homo sapiens,10GS.B
2,11GS,A,9606,Homo sapiens,11GS.A
3,11GS,B,9606,Homo sapiens,11GS.B
4,121P,A,9606,Homo sapiens,121P.A


## Filter the PDB chains by joining with the taxonomy data

In [10]:
pdb_filtered = pdb_map.merge(taxonomy, left_on=['structureId','chainId'], right_on=['pdbid','chain'], how='inner')
pdb_filtered = pdb_filtered.drop(['pdbid','chain'], axis=1)  # remove redundant columns
pdb_filtered.head()

,structureId,chainId,pdbPosition,pdbAminoAcid,refGenome,variationId,tax_id,scientific_name,structureChainId
0,1GUH,A,83,G,hgvs-grch37,chr6:g.52619766C>T,9606,Homo sapiens,1GUH.A
1,1XZ7,B,108,N,hgvs-grch37,chr11:g.5246945G>T,9606,Homo sapiens,1XZ7.B
2,2VCT,D,83,G,hgvs-grch37,chr6:g.52619766C>T,9606,Homo sapiens,2VCT.D
3,3KMF,G,708,N,hgvs-grch37,chr11:g.5246945G>T,9606,Homo sapiens,3KMF.G
4,1GSF,D,83,G,hgvs-grch37,chr6:g.52619766C>T,9606,Homo sapiens,1GSF.D


## Save mappings

In [11]:
pdb_filtered.to_csv(output_file_name, index=False)

## View mutations grouped by protein chain
Use the slider to view each protein chain.

In [12]:
chains = pdb_filtered.groupby('structureChainId')
print("Chains:", list(chains.groups.keys()))

Chains: ['1DXT.B', '1GSE.B', '1GSF.D', '1GUH.A', '1K3L.A', '1K3Y.B', '1OPL.A', '1PKW.A', '1PL1.A', '1QXD.D', '1VWT.D', '1XXT.D', '1XY0.B', '1XYE.B', '1XZ2.D', '1XZ4.B', '1XZ5.B', '1XZ7.B', '1XZU.D', '1XZV.B', '1Y09.D', '1Y0A.B', '1Y0C.B', '1Y0D.B', '1Y8W.B', '1YDZ.B', '1YH9.B', '1YHE.D', '1YHR.B', '1YZI.B', '2F4J.A', '2FO0.A', '2G1T.C', '2G2F.A', '2G2H.A', '2GQG.B', '2HIW.A', '2HZI.A', '2R6K.A', '2V7A.A', '2VCT.D', '2W6V.D', '2WJU.G', '3KMF.C', '3KMF.G', '3KTL.A', '3QRI.A', '3QRK.A', '3UE4.A', '4ACS.D', '4TWP.B', '4WA9.B', '4XS0.B', '4ZOG.A', '5JCU.C', '5MO4.A', '6ATO.A', '6ATP.A', '6ATQ.A', '6ATR.A', '6FQF.D']


In [13]:
def view_grouped_mutations(grouped_df, *args):
    chainIds = list(grouped_df.groups.keys())
    print(chainIds)

    def view3d(show_bio_assembly=False, show_surface=False, show_labels=True, i=0):
        group = grouped_df.get_group(chainIds[i])
        
        pdb_id, chain_id = chainIds[i].split('.')
        viewer = py3Dmol.view(query='pdb:' + pdb_id, options={'doAssembly': show_bio_assembly})

#       # polymer style
        viewer.setStyle({'cartoon': {'colorscheme': 'chain', 'width': 0.6, 'opacity':0.9}})

#       # non-polymer style
        viewer.setStyle({'hetflag': True}, {'stick':{'radius': 0.3, 'singleBond': False}})
       
        # highlight chain of interest in blue
        viewer.setStyle({'chain': chain_id},{'cartoon': {'color': 'blue'}})
        
        rows = group.shape[0]
        for j in range(0, rows):
            res_num = str(group.iloc[j]['pdbPosition'])
            mod_res = {'resi': res_num, 'chain': chain_id} 
            col = 'red'
            c_col = col + 'Carbon'
            viewer.addStyle(mod_res, {'stick':{'colorscheme':c_col, 'radius': 0.2}})
            viewer.addStyle(mod_res, {'sphere':{'color':col, 'opacity': 0.6}})          
            if show_labels:
                label = group.iloc[j]['variationId']
                viewer.addLabel(label, {'fontSize':7,'fontColor': 'black','backgroundColor':'ivory'}, {'resi': res_num, 'chain': chain_id})
        
        
        #print header
        print("PDB Id: " + pdb_id + " chain Id: " + chain_id)
        
        # print any specified additional columns from the dataframe
        for a in args:
            print(a + ": " + df.iloc[i][a])

        viewer.zoomTo({'chain': chain_id})
        
        if show_surface:
             viewer.addSurface(py3Dmol.SES,{'opacity':0.8,'color':'lightblue'},{'chain': chain_id})
                
        return viewer.show()
       
    s_widget = IntSlider(min=0, max=len(chainIds)-1, description='Structure', continuous_update=False)
    
    return interact(view3d, show_bio_assembly=False, show_surface=False, show_labels=True, i=s_widget)

Turn off scrolling in the viewer cell below (in Jupyter Notebook)

In [14]:
#%%javascript 
#IPython.OutputArea.prototype._should_scroll = function(lines) {return false;}

In [15]:
view_grouped_mutations(chains);

PDB Id: 1DXT chain Id: B


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## View one mutation at a time
Use the slider to view each mutation. Surrounding residues within the `distance_cutoff` are rendered as orange sticks.

In [16]:
def view_single_mutation(df, distance_cutoff, *args):

    def view3d(show_bio_assembly=False, show_surface=False, show_labels=True, i=0):        
        pdb_id, chain_id = df.iloc[i]['structureChainId'].split('.')
        
        viewer = py3Dmol.view(query='pdb:' + pdb_id, options={'doAssembly': show_bio_assembly})

        # polymer style
        viewer.setStyle({'cartoon': {'colorscheme': 'chain', 'width': 0.6, 'opacity':0.9}})

        # non-polymer style
        viewer.setStyle({'hetflag': True}, {'stick':{'radius': 0.3, 'singleBond': False}})
       
        # highlight chain of interest in green
        viewer.setStyle({'chain': chain_id},{'cartoon': {'color': 'blue', 'opacity':0.5}})
        
        # 
        res_num = str(df.iloc[i]['pdbPosition'])
        label = df.iloc[i]['variationId']
        mod_res = {'resi': res_num, 'chain': chain_id} 
        col = 'red'
        c_col = col + 'Carbon'
        viewer.addStyle(mod_res, {'stick':{'colorscheme':c_col, 'radius': 0.2}})
        viewer.addStyle(mod_res, {'sphere':{'color':col, 'opacity': 0.8}})          
        
        if show_labels:
            viewer.addLabel(label, {'fontSize':12,'fontColor': 'black','backgroundColor':'ivory'}, {'resi': res_num, 'chain': chain_id})

        # select neigboring residues by distance
        surroundings = {'chain': chain_id, 'resi': res_num, 'byres': True, 'expand': distance_cutoff}
        
        # residues surrounding mutation positions
        viewer.addStyle(surroundings,{'stick':{'colorscheme':'orangeCarbon', 'radius': 0.15}})
        
        viewer.zoomTo(surroundings)
        
        if show_surface:
             viewer.addSurface(py3Dmol.SES, {'opacity':0.8,'color':'lightblue'}, {'chain': chain_id})
                
         #print header
        print("PDB Id:", pdb_id, "chain Id:" , chain_id, "residue:", res_num, "mutation:", label)
        
        # print any specified additional columns from the dataframe
        for a in args:
            print(a + ": " + df.iloc[i][a])
            
        return viewer.show()
       
    s_widget = IntSlider(min=0, max=len(df)-1, description='Structure', continuous_update=False)
    
    return interact(view3d, show_bio_assembly=False, show_surface=False, show_labels=True, i=s_widget)

In [17]:
view_single_mutation(pdb_filtered, distance_cutoff);

PDB Id: 1GUH chain Id: A residue: 83 mutation: chr6:g.52619766C>T


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [18]:
# Shutdown Spark
spark = SparkSession.builder.getOrCreate().stop()

## Now run the next step
Map mutations occuring at protein-polymer interfaces: [3-MapToPolymerInteractions.ipynb](3-MapToPolymerInteractions.ipynb)